In [1]:
import os
import pandas as pd
import numpy as np

# Set TTM
ttm = 27

# Create directory for output if it doesn't exist
output_dir = "RiskPremia/Tau-independent/unique/moneyness_step_0d01/multivariate_clustering_9_27_45/Bitcoin_Premium/"
os.makedirs(output_dir, exist_ok=True)

# Load data
daily_price = pd.read_csv("Data/BTC_USD_Quandl_2011_2023.csv", parse_dates=["Date"])
daily_price = daily_price.sort_values("Date")

# Filter data from 2014 onwards
daily_price = daily_price[daily_price["Date"] >= "2014-01-01"]

# Convert prices to numeric
daily_price["Adj.Close"] = pd.to_numeric(daily_price["Adj.Close"], errors='coerce')

# 27-day realized returns
S_t_plus_ttm = daily_price["Adj.Close"].iloc[ttm:].values
S_t = daily_price["Adj.Close"].iloc[:-ttm].values
daily_price["return_t_minus_27"] = np.concatenate(([np.nan] * 27, (S_t_plus_ttm - S_t) / S_t))
daily_price["return_t_plus_27"] = np.concatenate(((S_t_plus_ttm - S_t) / S_t, [np.nan] * 27))

# 1-day realized returns
S_t_plus_1 = daily_price["Adj.Close"].iloc[1:].values
S_t = daily_price["Adj.Close"].iloc[:-1].values
daily_price["return_t_minus_1"] = np.concatenate(([np.nan], (S_t_plus_1 - S_t) / S_t))
daily_price["return_t_plus_1"] = np.concatenate(((S_t_plus_1 - S_t) / S_t, [np.nan]))

# Drop NaN values
daily_price = daily_price.dropna()

# Load cluster dates
common_dates = pd.read_csv(os.path.join('Clustering', 'Tau-independent', 'unique', 'moneyness_step_0d01', 'multivariate_clustering_9_27_45',
                                        'common_dates_cluster.csv'))

dates_Q = {
    0: common_dates[common_dates["Cluster"] == 0]["Date"].astype(str).tolist(),
    1: common_dates[common_dates["Cluster"] == 1]["Date"].astype(str).tolist()
}

# Realized variance
realized_vola = np.full((len(daily_price), 2), np.nan)

dates = daily_price["Date"].values
for i in range(27, len(dates) - 27):
    sp1 = daily_price.sort_values("Date")
    
    # Filter using datetime operations
    St_minus_tau = sp1[(sp1["Date"] >= dates[i] - pd.Timedelta(days=ttm)) & 
                        (sp1["Date"] <= dates[i])]["Adj.Close"].values
    St_plus_tau = sp1[(sp1["Date"] >= dates[i]) & 
                       (sp1["Date"] <= dates[i] + pd.Timedelta(days=ttm))]["Adj.Close"].values
    
    simpleret_before = (St_minus_tau[1:] / St_minus_tau[:-1]) - 1 if len(St_minus_tau) > 1 else []
    simpleret_after = (St_plus_tau[1:] / St_plus_tau[:-1]) - 1 if len(St_plus_tau) > 1 else []
    
    realized_vola[i, 0] = np.sqrt(np.sum(np.square(simpleret_before))) if simpleret_before.size>0 else np.nan
    realized_vola[i, 1] = np.sqrt(np.sum(np.square(simpleret_after))) if simpleret_after.size>0 else np.nan

daily_price["simpleRV"] = realized_vola[:, 0]
daily_price["simpleFV"] = realized_vola[:, 1]

# Get index
idx_HV = daily_price[daily_price["Date"].astype(str).isin(dates_Q[0])].index
idx_LV = daily_price[daily_price["Date"].astype(str).isin(dates_Q[1])].index

RR_OA = daily_price.loc[idx_HV.union(idx_LV)].sort_values("Date")
RR_HV = daily_price.loc[idx_HV].sort_values("Date")
RR_LV = daily_price.loc[idx_LV].sort_values("Date")

# Save results
RR_OA.to_excel(os.path.join(output_dir, "477_sample_return_OA.xlsx"), index=False)
RR_HV.to_excel(os.path.join(output_dir, "477_sample_return_HV.xlsx"), index=False)
RR_LV.to_excel(os.path.join(output_dir, "477_sample_return_LV.xlsx"), index=False)